### All needed imports

In [37]:
import os
import cv2
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Add, Activation, Flatten, concatenate, Lambda


### Limit GPU Usage

In [38]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


### Preprocessing Data for Model 1

#### Video Paths

In [39]:
falling_paths = [

                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
                 ]

 

default_paths = [

                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default16.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default17.mp4",
                #  "./../datasets/vids/splitted/new_still/resized_logitech-default18.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default19.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default20.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default21.mp4"
                 ]

test_falling_paths = [
                "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_default_paths = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"
]

#### Processing code, returns frame (num_frames, 224,224,3) and frame_diffs (num_frames, 224,224)

In [40]:
# Code to apply the same random transformation 

In [41]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_BGR2GRAY)

            if prev_gray_frame is None:
                frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
            else:
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_diff)
            labels.append(label)  # 0 for still, 1 for moving
        # print(len(labels))
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [42]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    # frame_diffs dimension
    # print(frame_diffs.shape)
    frame_diffs_expanded = np.expand_dims(frame_diffs, axis=-1) # Add an extra dimension
    # frame_diffs_expanded dimension
    # print(frame_diffs_expanded.shape)
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1) # Concatenate along the last axis
    # combined_input dimension
    # print(combined_input.shape)
    
    # Visualize the difference
    # cv2.imwrite('img_still.png', combined_input[10])
    return combined_input

In [43]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
# test_still_frames, test_still_diff, test_still_labels = process_videos(test_default_paths, 0)
# print(still_frames.shape)
falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)
# test_falling_frames, test_falling_diff, test_falling_labels = process_videos(test_falling_paths, 1)
# print(falling_frames.shape)

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
# print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
# print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
# print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)
# print(combined_input)

# Combine them
# test_concatenate_frames = np.concatenate([test_still_frames, test_falling_frames], axis = 0)
# test_concatenate_diff =  np.concatenate([test_still_diff, test_falling_diff], axis = 0)
# test_concatenate_labels =  np.concatenate([test_still_labels, test_falling_labels], axis = 0)
# print(test_concatenate_frames.shape)
# Create 224x224x4 shape for the model
# test_combined_input = combine_frames_and_diffs(test_concatenate_frames, test_concatenate_diff)
print(combined_input.shape)
# print(test_combined_input.shape)
# Shuffle the data if needed
indices = np.arange(combined_input.shape[0])
np.random.shuffle(indices)
combined_input = combined_input[indices]
print(combined_input.dtype)
concatenate_labels = concatenate_labels[indices]

#data augmentation 
data_augmentation = tf.keras.Sequential([
    # layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast(factor=0.1)

])


(2495, 224, 224, 4)
float32


In [44]:
# Create a TensorFlow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)
    # test_dataset = tf.data.Dataset.from_tensor_slices((test_combined_input, test_concatenate_labels))
    # test_dataset = test_dataset.shuffle(buffer_size=len(test_combined_input))
    # test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split the dataset into training and validation
train_dataset = dataset.take(int(0.85 * len(dataset)))
val_dataset = dataset.skip(int(0.85 * len(dataset)))

# dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
# Batch and prefetch if needed
# dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

#### Create the model

In [45]:
def create_model():
    combined_input = Input(shape=(224, 224, 4), name='combined_input')
    
    # Splitting the input into RGB and Difference channels
    rgb_input = Lambda(lambda x: x[:, :, :, :3])(combined_input)  # First 3 channels
    diff_input = Lambda(lambda x: x[:, :, :, 3:4])(combined_input)  # Last channel
    # print(rgb_input.shape)
    # print(diff_input.shape)

    # Processing the RGB channels
    x_rgb = Conv2D(16, (3, 3), activation='relu')(rgb_input)
    x_rgb = MaxPooling2D((2, 2))(x_rgb)
    x_rgb = Conv2D(32, (3, 3), activation='relu')(x_rgb)
    x_rgb = MaxPooling2D((2, 2))(x_rgb)
    x_rgb = GlobalAveragePooling2D()(x_rgb)
    
    # Processing the Difference channel
    x_diff = Conv2D(16, (3, 3), activation='relu')(diff_input)
    x_diff = MaxPooling2D((2, 2))(x_diff)
    x_diff = GlobalAveragePooling2D()(x_diff)
    
    # Combine the processed outputs from the two paths
    combined_features = concatenate([x_rgb, x_diff])
    
    # Fully connected layers for classification
    x = Dense(32, activation='relu')(combined_features)
    output = Dense(1, activation='sigmoid')(x)  # Single output with sigmoid activation

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
    
    
model = create_model()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 combined_input (InputLayer)    [(None, 224, 224, 4  0           []                               
                                )]                                                                
                                                                                                  
 lambda_4 (Lambda)              (None, 224, 224, 3)  0           ['combined_input[0][0]']         
                                                                                                  
 conv2d_6 (Conv2D)              (None, 222, 222, 16  448         ['lambda_4[0][0]']               
                                )                                                                 
                                                                                            

In [46]:
checkpoint_filepath = 'tmp/checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [47]:
model.fit(train_dataset, validation_data=val_dataset, epochs=15, callbacks=[model_checkpoint_callback])
model.load_weights(checkpoint_filepath)

Epoch 1/15
132/132 [==============================] - 5s 35ms/step - loss: 0.5881 - accuracy: 0.7827 - val_loss: 0.6426 - val_accuracy: 0.5352
Epoch 2/15
132/132 [==============================] - 4s 33ms/step - loss: 0.3666 - accuracy: 0.8556 - val_loss: 0.2965 - val_accuracy: 0.8930
Epoch 3/15
132/132 [==============================] - 5s 35ms/step - loss: 0.3204 - accuracy: 0.8741 - val_loss: 0.2938 - val_accuracy: 0.8642
Epoch 4/15
132/132 [==============================] - 4s 34ms/step - loss: 0.2981 - accuracy: 0.8826 - val_loss: 0.2561 - val_accuracy: 0.8877
Epoch 5/15
132/132 [==============================] - 4s 34ms/step - loss: 0.2779 - accuracy: 0.8911 - val_loss: 0.2851 - val_accuracy: 0.8668
Epoch 6/15
132/132 [==============================] - 5s 35ms/step - loss: 0.2853 - accuracy: 0.8911 - val_loss: 0.2886 - val_accuracy: 0.8668
Epoch 7/15
132/132 [==============================] - 4s 34ms/step - loss: 0.2518 - accuracy: 0.9029 - val_loss: 0.2231 - val_accuracy: 0.8930

#### Save the model

In [48]:
# model.evaluate(test)
# model.save("m1")

#### Quantize and save as tflite

In [49]:
def representative_dataset():
    for data, label in tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels)).batch(1).take(100):
        yield [tf.dtypes.cast(data, tf.float32)]

with tf.device('/cpu:0'):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_quant_m1 = converter.convert()

# Save the quantized model 1
with open('m1_quant_v5.tflite', 'wb') as f: f.write(tflite_quant_m1)

INFO:tensorflow:Assets written to: /tmp/tmp9a2jbpw_/assets


INFO:tensorflow:Assets written to: /tmp/tmp9a2jbpw_/assets
/home/sylvia/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-15 14:47:34.721255: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-15 14:47:34.721276: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-15 14:47:34.721419: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp9a2jbpw_
2023-08-15 14:47:34.723005: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-15 14:47:34.723016: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmp9a2jbpw_
2023-08-15 14:47:34.727655: I tensorflow/cc/saved_model/loader.cc:

#### Pass in video for model 1 to predict to generate data for model 2

In [50]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="m1_quant.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
print('Input shape:', input_details[0]['shape'])
output_details = interpreter.get_output_details()
print('Output shape:', output_details[0]['shape'])

# [1,1] u get [[1]], instead of [1]
# idk why it wraps an extra bracket but it just does

Input shape: [  1 224 224   4]
Output shape: [1 1]


#### Predict with model 1 and save the results into an array

In [51]:
def process_single_vid(path):
    # know which video I am processing
    print(path)
    frames = []
    frame_diffs = []
    video_cap = cv2.VideoCapture(path)
    
    prev_gray_frame = None
    
    while video_cap.isOpened():
        ret, frame = video_cap.read()
        
        if not ret:
            break
        
        # Resize and convert frame to RGB
        frame_resized = cv2.resize(frame, (224, 224))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        
        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_BGR2GRAY)

        if prev_gray_frame is None:
            frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
        else:
            frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
            # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
        
        prev_gray_frame = gray_frame
        
        frames.append(frame_rgb)
        frame_diffs.append(frame_diff)

    video_cap.release()
    return np.array(frames), np.array(frame_diffs)

def predict_single_vid(interpreter, input_data):
    # Get the input and output details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Initialize an empty list to store the results
    results = []

    # Iterate over each instance in the input data
    for instance in input_data:
        # Set the tensor to point to the input data to be inferred
        interpreter.set_tensor(input_details['index'], np.expand_dims(instance, axis=0))

        # Run the computation
        interpreter.invoke()

        # Get the output tensor
        output = interpreter.get_tensor(output_details['index'])

        # Append the output to the results list
        results.append(output)

    # Convert the results list to an array and return it
    return np.array(results)

def add_noise(data, noise_level=5):
    # Convert to float
    float_data = data.astype(np.float32)
    
    # Generate random noise
    noise = np.random.uniform(-noise_level, noise_level, data.shape)
    
    # Add the noise to the data
    noisy_data = float_data + noise
    
    # Clip values to uint8 range and convert back to uint8
    noisy_data = np.clip(noisy_data, 0, 255).astype(np.uint8)
    
    return noisy_data

In [52]:
result_8 = []
labels = []
# for vid in falling_paths:
#     vid_frames, vid_diffs = process_single_vid(vid)
#     # reshape and combine inputs
#     combined = combine_frames_and_diffs(vid_frames, vid_diffs)
#     # convert datatype from float 32 to uint8
#     combined = combined.astype(np.uint8)
#     # print(combined)
#     # predict_single_vid
#     res = predict_single_vid(interpreter, combined)
#     # save results to array in sliding window fashion 8 at a time
#     for i in range(len(res) - 8 + 1):
#         window = res[i:i+8]
#         result_8.append(window)
#         # print(result_8)
#         labels.append(1)

for vid in default_paths:
    vid_frames, vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    combined = combine_frames_and_diffs(vid_frames, vid_diffs)
    # convert datatype from float 32 to uint8
    combined = combined.astype(np.uint8)
    # predict_single_vid
    res = predict_single_vid(interpreter, combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(res) - 8 + 1):
        window = res[i:i+8]
        result_8.append(window)
        labels.append(0)

./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4
./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4
./../datasets/vids/splitted/new_still/resiz

In [53]:

result_8 = np.array(result_8)
labels = np.array(labels)

result_8 = result_8.reshape(len(result_8), 8)
noisy_data = add_noise(result_8)
doubled_data = np.vstack((result_8, noisy_data))
augmented_labels = labels
doubled_labels = np.hstack((labels, augmented_labels))

print(doubled_data.shape)
print(doubled_labels.shape)

np.savetxt('model2data_augmented.txt', doubled_data, fmt='%d', delimiter=' ')
np.savetxt('model2labels_augmented.txt', doubled_labels, fmt='%d', delimiter=' ')

(2656, 8)
(2656,)


In [54]:
X = np.loadtxt('model2data_augmented.txt', dtype=np.float32, delimiter=' ')
X = X/255.0

print(X.shape)
Y = np.loadtxt('model2labels_augmented.txt', dtype=np.float32, delimiter=' ')
# print(Y.shape)

# fall_X = np.loadtxt('./../K-optimized1-training/model1_output_fall.txt', dtype=np.uint8, delimiter=' ')
# default_X = np.loadtxt('./../K-optimized1-training/model1_output_default.txt', dtype=np.uint8, delimiter=' ')
# X = np.concatenate((fall_X, default_X))

# fall_Y = np.ones(len(fall_X))
# default_Y = np.zeros(len(default_X))

# Y = np.concatenate((fall_Y, default_Y))
# print(Y.shape)

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)



(2656, 8)


#### Prepare testing data

In [55]:
# test_result_8 = []
# test_labels = []

# for vid in test_falling_paths:
#     test_vid_frames, test_vid_diffs = process_single_vid(vid)
#     # reshape and combine inputs
#     test_combined = combine_frames_and_diffs(test_vid_frames, test_vid_diffs)
#     # convert datatype from float 32 to uint8
#     test_combined = test_combined.astype(np.uint8)
#     # print(combined)
#     # predict_single_vid
#     test_res = predict_single_vid(interpreter, test_combined)
#     # save results to array in sliding window fashion 8 at a time
#     for i in range(len(test_res) - 8 + 1):
#         window = test_res[i:i+8]
#         test_result_8.append(window)
#         test_labels.append(1)

# for vid in test_default_paths:
#     test_vid_frames, test_vid_diffs = process_single_vid(vid)
#     # reshape and combine inputs
#     test_combined = combine_frames_and_diffs(test_vid_frames, test_vid_diffs)
#     # convert datatype from float 32 to uint8
#     test_combined = test_combined.astype(np.uint8)
#     # predict_single_vid
#     test_res = predict_single_vid(interpreter, test_combined)
#     # save results to array in sliding window fashion 8 at a time
#     for i in range(len(test_res) - 8 + 1):
#         window = test_res[i:i+8]
#         test_result_8.append(window)
#         test_labels.append(0)

# test_result_8 = np.array(test_result_8)
# test_labels = np.array(test_labels)


In [56]:
# print(test_result_8.shape)
# print(test_labels.shape)
# test_result_8 = test_result_8.reshape(len(test_result_8), 8)
# np.savetxt('model2data_test.txt', test_result_8, fmt='%d', delimiter=' ')
# np.savetxt('model2labels_test.txt', test_labels, fmt='%d', delimiter=' ')
# print(test_result_8.shape)

In [67]:
# test_X = np.loadtxt('model2data_test.txt', dtype=np.float32, delimiter=' ')
# test_X = test_X/255.0
# print(test_X.shape)
# test_Y = np.loadtxt('model2labels_test.txt', dtype=np.float32, delimiter=' ')
# print(test_Y.shape)

# test_fall_X = np.loadtxt('./../K-optimized1-training/model1_output_fall_test.txt', dtype=np.uint8, delimiter=' ')
# test_default_X = np.loadtxt('./../K-optimized1-training/model1_output_default_test.txt', dtype=np.uint8, delimiter=' ')
# test_X = np.concatenate((test_fall_X, test_default_X))

# test_fall_Y = np.ones(len(test_fall_X))
# test_default_Y = np.zeros(len(test_default_X))

# test_Y = np.concatenate((test_fall_Y, test_default_Y))
# print(test_Y.shape)

# Create the dataset
# test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_Y))
# test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)
# test_dataset = test_dataset.shuffle(buffer_size=len(test_combined_input))
# Split the dataset into training and validation
# test = test_dataset.take(int(0.8 * len(test_dataset)))
train_dataset = dataset.take(int(0.85 * len(dataset)))
val_dataset = dataset.skip(int(0.85 * len(dataset)))

#### Create model 2

In [66]:
def create_model_2():
    input_shape = (8, )  # Input shape for the sequence of 8 uint8 values
    
    # Define the model
    input_layer = Input(shape=input_shape, name='input_layer')
    
    # Several dense layers to extract features and relationships
    x = Dense(64, activation='relu')(input_layer)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(16, activation='relu')(x)
    
    # Output layer
    output = Dense(1, activation='sigmoid')(x)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model_2 = create_model_2()
model_2.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 8)]               0         
                                                                 
 dense_14 (Dense)            (None, 64)                576       
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_16 (Dense)            (None, 16)                528       
                                                                 
 dense_17 (Dense)            (None, 1)                 17  

#### Prepare Tensorflow dataset

In [68]:
model_2.fit(train_dataset, validation_data=val_dataset, epochs=15, callbacks=[model_checkpoint_callback])
model_2.load_weights(checkpoint_filepath)

Epoch 1/15
141/141 [==============================] - 1s 2ms/step - loss: 0.2283 - accuracy: 0.9552 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 2/15
141/141 [==============================] - 0s 3ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 6.5007e-05 - val_accuracy: 1.0000
Epoch 3/15
141/141 [==============================] - 1s 7ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.3724e-05 - val_accuracy: 1.0000
Epoch 4/15
141/141 [==============================] - 1s 8ms/step - loss: 6.4342e-04 - accuracy: 1.0000 - val_loss: 3.5119e-06 - val_accuracy: 1.0000
Epoch 5/15
141/141 [==============================] - 1s 8ms/step - loss: 3.0925e-04 - accuracy: 1.0000 - val_loss: 1.1974e-06 - val_accuracy: 1.0000
Epoch 6/15
141/141 [==============================] - 1s 7ms/step - loss: 1.7436e-04 - accuracy: 1.0000 - val_loss: 5.2052e-07 - val_accuracy: 1.0000
Epoch 7/15
141/141 [==============================] - 1s 8ms/step - loss: 9.8955e-05 - accuracy: 1.0000 - val_loss: 

In [60]:
# model_2.evaluate(test)

In [69]:
print(X.dtype)
def representative_dataset_2():
    for i in range(len(X)):
        sample = X[i:i+1].astype(np.float32)  # Extract and cast in one line
        yield [sample]

converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_2
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

# Save the model to disk
with open('m2_quant_v5.tflite', 'wb') as f: f.write(tflite_model)

float32
INFO:tensorflow:Assets written to: /tmp/tmpc8r77v6m/assets


INFO:tensorflow:Assets written to: /tmp/tmpc8r77v6m/assets
/home/sylvia/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-15 15:23:50.683831: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-15 15:23:50.683851: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-15 15:23:50.683990: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpc8r77v6m
2023-08-15 15:23:50.685096: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-15 15:23:50.685106: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpc8r77v6m
2023-08-15 15:23:50.688303: I tensorflow/cc/saved_model/loader.cc:

#### Checking input / output of model

In [62]:
# Load TFLite model and allocate tensors.
interpreter_2 = tf.lite.Interpreter(model_path="m2_quant_v5.tflite")
interpreter_2.allocate_tensors()

input_details = interpreter_2.get_input_details()
print('Input shape:', input_details[0]['shape'])
print('Input type:', input_details[0]['dtype'])
output_details = interpreter_2.get_output_details()
print('Output shape:', output_details[0]['shape'])
print('Output type:', output_details[0]['dtype'])

Input shape: [1 8]
Input type: <class 'numpy.uint8'>
Output shape: [1 1]
Output type: <class 'numpy.uint8'>
